In [3]:
import keras_cv
import keras
import tensorflow as tf
import os
import numpy as np
import tensorflow_datasets as tfds
import PIL

In [4]:
classes = ['nyathapola']
class_mapping = dict(zip(range(len(classes)), classes))

In [11]:
image_dir = "dataset/train/images"
annotation_dir = "dataset/train/labels"

In [40]:
def get_bboxes_classes(label_dir):
    classes_ = list()
    bboxes = list()

    for labels in os.listdir(label_dir):
        temp_c = []
        temp_b = []
        # num_boxes = 0
        with open(os.path.join(label_dir, labels)) as file:
            temp = file.readlines()
            for cords in temp:
                temp_c.append(float(cords.split(' ')[0]))
                # num_boxes += 1
                bbox = list(map(float, cords.split(' ')[1:]))
                # temp_b.append(float(num_boxes))
                temp_b.append(bbox)
        classes_.append(temp_c)
        bboxes.append(len(temp_b))

    bbox = tf.ragged.constant(bboxes, dtype = tf.float32)
    classes = tf.ragged.constant(classes_, dtype = tf.float32)

    return bbox, classes

In [15]:
bbox, classes = get_bboxes_classes(annotation_dir)
print(bbox)
print(classes)

tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1.], shape=(79,), dtype=float32)
<tf.RaggedTensor [[0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0]]>


In [41]:
def create_dataset(image_paths, bboxes, classes):
    return tf.data.Dataset.from_tensor_slices((image_paths, bboxes, classes))

In [42]:
def load_image_paths(img_dir):
    image_paths = list()

    for images in os.listdir(img_dir):
        image_paths.append(os.path.join(img_dir, images))
    
    # return tf.ragged.constant(image_paths)
    return tf.constant(image_paths)

In [43]:
image_paths = load_image_paths(image_dir)
bboxes, classes = get_bboxes_classes(annotation_dir)
loaded_dataset = create_dataset(image_paths, bboxes, classes)

In [44]:
train_data = loaded_dataset.take(int(len(os.listdir(annotation_dir)) * 0.8))
val_data = loaded_dataset.skip(int(len(os.listdir(annotation_dir)) * 0.8))

In [45]:
resizing = keras_cv.layers.JitteredResize(target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xywh")

In [46]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

In [47]:
def load_dataset(image_path, classes, bbox):
    image = load_image(image_path)
    # boxes = keras_cv.bounding_box.convert_format(bbox, images=image, source="xywh", target="xywh")
    boxes = tf.ragged.constant(bbox, dtype = tf.float32)
    
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": boxes,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}

In [48]:
train_set = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_set = train_set.shuffle(4 * 4)
# train_set = train_set.ragged_batch(4, drop_remainder=True)
train_set = train_set.batch(4, drop_reminder=True)

OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\acer\AppData\Local\Temp\ipykernel_14208\374089111.py", line 4, in load_dataset  *
        boxes = tf.ragged.constant(bbox, dtype = tf.float32)

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.


In [29]:
val_set = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_set = val_set.shuffle(4 * 4)
val_set = val_set.ragged_batch(4, drop_remainder=True)

In [32]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

In [33]:
train_set = train_set.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_set = train_set.prefetch(tf.data.AUTOTUNE)

In [34]:
val_set = val_set.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_set = val_set.prefetch(tf.data.AUTOTUNE)

In [35]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_s_backbone_coco")

In [36]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xywh",
    backbone=backbone,
    fpn_depth=1)

In [37]:
yolo.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, global_clipnorm=10.0),
             classification_loss="binary_crossentropy", 
             box_loss="ciou")

In [39]:
yolo.fit(train_set, epochs=3)

Epoch 1/3


TypeError: Exception encountered when calling Pad.call().

[1mFailed to convert elements of tf.RaggedTensor(values=tf.RaggedTensor(values=Tensor("yolov8_detector_1/functional_1_1/rescaling_1/Add:0", shape=(None, 3), dtype=float32), row_splits=Tensor("data_2:0", shape=(None,), dtype=int64)), row_splits=Tensor("data_1:0", shape=(5,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.[0m

Arguments received by Pad.call():
  • x=tf.Tensor(shape=(4, None, None, 3), dtype=float32)
  • constant_values=None